# Backmap a MARTINI CG structure
To obtain an atomistic structure from coarse grained simulations we need a CG structure and a topology file for the atomistic system.

In [ ]:
import running_rabbit as rr

In [ ]:
# input
cg_structure = "2xUBQ_cg.pdb"            # coarse grained structure for backmapping
aa_topology  = "2xUBQ_aa.top"            # atomistic topology

# output
aa_structure_raw = "2xUBQ_aa_raw.gro"    # initial structure after running backward.py
aa_structure     = "2xUBQ_aa.gro"        # final structure after energy min.
path_em          = "./em_after_backmap"

In [ ]:
# HOTFIX, convert into package at some point?
backward = "python ../../backward.py"

In [ ]:
# perform backmapping
# backward script has some bugs and will give some error warnings for lipids
# should not be a problem for proteins
!{backward} -f {cg_structure} -p {aa_topology} -o {aa_structure_raw} -to gromos -kick 0.05

The atomistic structure after backmapping is highly unphysical and has to be energy minimized to obtain a proper structure. Therefore, we perform energy minimization in three steps:
1. Backbone position restrained, small em step, higher force threshold, no bond constraints 
2. Backbone position restrained, bigger em step, lower force threshold, no bond constraints
3. no position restraints, bigger em step, lower force threshold, bonds constrained with LINKS 

In [ ]:
# note: this is a special running rabbit template for backmapping
rabbit              = rr.Rabbit(ff="gromos54a7", template_name="backmap_em")
rabbit.structure    = aa_structure_raw
rabbit.topology     = aa_topology
rabbit.destination  = path_em
rabbit.run()

In [ ]:
# run EM
!cd em_after_backmap && bash setup

In [ ]:
# copy final structure
!cp {path_em}/eq5.gro {aa_structure}

In [ ]:
# note: some atoms are not in the first unit cell after mackmapping
# resolve by:
!gmx editconf -f {aa_structure} -o {aa_structure} -pbc yes